# <u>Training Exercices - Endogeneity: Application<u>

__<u>Jorge Pineño Pérez<u>__ / Module 5 - Endogeneity

In [1]:
# package imports
install.packages("readxl")
install.packages("dplyr")
library(readxl)
library(dplyr)

Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'readxl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpMdPOff\downloaded_packages


Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpMdPOff\downloaded_packages



Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




__<u>Training Exercise 4.5<u>__

In [2]:
# import the Engineering MOOC dataframe
mooc <- read_excel('TrainExer45.xls')
head(mooc)

GPA,PARTICIPATION,GENDER,EMAIL
<dbl>,<dbl>,<dbl>,<dbl>
7.386898,1,0,1
6.493597,1,1,1
4.956625,0,1,1
7.442979,1,1,0
6.281521,0,1,1
5.391270,0,0,1


__(a)__

In [3]:
# regress GPA on a constant, "GENDER", and "PARTICIPATION", and check that we get
# R-squared = 0.24, and a t-statistic of 17.59 for the coefficient of "PARTICIPATION"
gpa_ols <- lm(GPA ~ GENDER + PARTICIPATION, data=mooc)
summary(gpa_ols)


Call:
lm(formula = GPA ~ GENDER + PARTICIPATION, data = mooc)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.00981 -0.48166  0.01022  0.47071  1.80200 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    5.77111    0.03397 169.874  < 2e-16 ***
GENDER        -0.21376    0.04431  -4.824 1.63e-06 ***
PARTICIPATION  0.82437    0.04686  17.593  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.6985 on 997 degrees of freedom
Multiple R-squared:  0.2441,	Adjusted R-squared:  0.2426 
F-statistic:   161 on 2 and 997 DF,  p-value: < 2.2e-16


__(b)__

In [4]:
# perform the first-stage regression of "PARTICIPATION" on "GENDER" and "EMAIL"
part_fs <- lm(PARTICIPATION ~ GENDER + EMAIL, data=mooc)
summary(part_fs)


Call:
lm(formula = PARTICIPATION ~ GENDER + EMAIL, data = mooc)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.5625 -0.1496 -0.1011  0.4375  0.8989 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.10112    0.02290   4.415 1.12e-05 ***
GENDER       0.04846    0.02690   1.801   0.0719 .  
EMAIL        0.41290    0.02690  15.349  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4246 on 997 degrees of freedom
Multiple R-squared:  0.1956,	Adjusted R-squared:  0.194 
F-statistic: 121.2 on 2 and 997 DF,  p-value: < 2.2e-16


In [5]:
# add the residuals of "gpa_ols" and the fitted values of "part_fs"
# to the tibble to be able to perform the second-stage regression
mooc <- mooc %>% mutate(fitted_part = fitted(part_fs), residuals_gpa = resid(gpa_ols))
head(mooc)

GPA,PARTICIPATION,GENDER,EMAIL,fitted_part,residuals_gpa
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7.386898,1,0,1,0.5140228,0.7914183
6.493597,1,1,1,0.5624784,0.1118752
4.956625,0,1,1,0.5624784,-0.6007279
7.442979,1,1,0,0.1495790,1.0612579
6.281521,0,1,1,0.5624784,0.7241681
5.391270,0,0,1,0.5140228,-0.3798420


In [13]:
# regress gpa on a constant, "GENDER", and "fitted_part", and check that
# we get R-squared = 0.013, and the coefficient of "fitted_part" = 2.24
gpa_2SLS <- lm(GPA ~ GENDER + fitted_part, data=mooc)
summary(gpa_2SLS)


Call:
lm(formula = GPA ~ GENDER + fitted_part, data = mooc)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.07155 -0.57838 -0.05513  0.59162  2.23636 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.94787    0.05117 116.239  < 2e-16 ***
GENDER      -0.17276    0.05121  -3.373 0.000771 ***
fitted_part  0.24050    0.12246   1.964 0.049819 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.7981 on 997 degrees of freedom
Multiple R-squared:  0.01325,	Adjusted R-squared:  0.01127 
F-statistic: 6.694 on 2 and 997 DF,  p-value: 0.001294
